In [1]:
import sys
import re
import pandas as pd
import json
import os
import dotenv

from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [5]:
# All functions
def split_and_clean_documents(docs, page=0, chunk_size=1024, chunk_overlap=0):
    text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(docs)
    
    table = []

    for i in range(len(texts)):
        cleaned_content = re.sub(r'[\u202a-\u202e]', '', texts[i].page_content)
        cleaned_content = re.sub(r'^\s*(?!\d+\.\d+\.\d+)[๐-๙0-9]+[\s.-]*', '', cleaned_content)
        cleaned_content = re.sub(r'^\s*-\s*\d+\s*-\s*', '', cleaned_content)
        cleaned_content = cleaned_content.strip()
        #texts[i] = Document(metadata=texts[i].metadata, page_content=cleaned_content)

        source = texts[i].metadata.get("source")
        source = re.sub(r'^.*?\\', '', source)
        page = texts[i].metadata.get("page")
        total = texts[i].metadata.get("total_pages")
        table.append({
            "Source":source,
            "Content":cleaned_content,
            "Page":page+1,
            "Total_pages":total,
        })
    
    return pd.DataFrame(table)

In [2]:
# Load docs
doc_paths = [
    #"./data\Final_หลักสูตรวท.บ. (วิทยาศาสตร์และนวัตกรรมข้อมูล) (พหุ) (ปป. 66) (2).pdf",
    "./data\วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf"
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    try:
        if str(file_path).endswith(".pdf"):
            loader = PyMuPDFLoader(file_path=file_path)
            data = loader.load_and_split()
        else:
            print('Upload a file successful')
            sys.exit()

        docs.extend(data)

    except Exception as e:
        print(f"Error loading document {file_name}: {e}")

In [3]:
len(docs)

182

In [6]:
df = split_and_clean_documents(docs=docs)

In [7]:
df

,Source,Content,Page,Total_pages
0,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หลักสูตรวิทยาศาสตรบัณฑิต \nสาขาวิชาวิทยาการคอม...,1,205
1,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,รายละเอียดของหลักสูตร \nหลักสูตรวิทยาศาสตรบัณฑ...,2,205
2,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,1.4.2 ประเภทของหลักสูตร \n \n R \nหลักสูตรปริ...,3,205
3,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,1.5 อาชีพที่สามารถประกอบไดหลังสำเร็จการศึกษา ...,4,205
4,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวดที่ 2 คุณสมบัติผูเข้าศึกษา \n2.1 การรับเ...,5,205
...,...,...,...,...
177,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,ข้อ ๕6 ให้นับรวมเวลาในระหว่างที่ถูกลงโทษให้พัก...,179,205
178,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,ตามหลักเกณฑ์การวัดผลของหลักสูตรที่จะย้ายเข้า เ...,180,205
179,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวด ๑๖ \nการได้รับปริญญาเกียรตินิยม \n-------...,181,205
180,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวด ๑๗ \nการเสนอชื่อและการอนุมัติอนุปริญญาหรื...,182,205


In [8]:
df.to_csv("SampleData_ComSci.csv", index=False ,encoding='utf-8-sig')

In [113]:
for i in range(len(texts)):
    cleaned_content = re.sub(r'[\u202a-\u202e]', '', texts[i].page_content)
    cleaned_content = re.sub(r'^\s*(?!\d+\.\d+\.\d+)[๐-๙0-9]+[\s.-]*', '', cleaned_content)
    cleaned_content = re.sub(r'^\s*-\s*\d+\s*-\s*', '', cleaned_content)
    cleaned_content = cleaned_content.strip()
    texts[i] = Document(metadata=texts[i].metadata, page_content=cleaned_content)

print(texts[80])

page_content='“ภาคการศึกษา” หมายความว่า ภาคการศึกษาในระบบทวิภาค หรือในระบบไตรภาค 
แต่ไม่รวมภาคฤดูร้อน 
“สถาบันอุดมศึกษาอื่น” หมายความว่า มหาวิทยาลัยหรือสถาบันอุดมศึกษาอื่นที่ได้รับ
การจัดตั้งตามกฎหมายของไทยหรือเป็นมหาวิทยาลัยหรือสถาบันอุดมศึกษาของต่างประเทศ  
“ข้อกําหนดหลักสูตร” หมายความว่า ข้อกําหนดที่ว่าด้วยหลักเกณฑ์และเงื่อนไข 
ในการศึกษาของหลักสูตรตามที่ได้รับอนุมัติจากสภามหาวิทยาลัย 
“นายทะเบียน” หมายความว่า ผู้อํานวยการสํานักงานทะเบียนนักศึกษา 
ข้อ ๕ ให้อธิการบดีเป็นผู้รักษาการตามข้อบังคับนี้ และให้มีอํานาจออกประกาศของ
มหาวิทยาลัยเพื่อปฏิบัติตามข้อบังคับนี้ได้  
หมวด ๑ 
ระบบการจัดการศึกษา และอาจารย์ที่ปรึกษา 
------------------------- 
ข้อ ๖ มหาวิทยาลัยจัดการศึกษาด้วยหลักการประสานงานด้านวิชาการระหว่างคณะ คณะใด
มีหน้าที่จัดการศึกษาในวิชาการด้านใดให้จัดการศึกษาในวิชาการด้านนั้นแก่นักศึกษาของมหาวิทยาลัย
ทุกคณะ เว้นแต่กรณีที่มีเหตุผลอันสมควรและได้รับความเห็นชอบจากสภามหาวิทยาลัย 
ข้อ ๗ การจัดการศึกษาของมหาวิทยาลัยมี 2 ระบบ ดังนี้   
(๑) ระบบทวิภาคเป็นการจัดการศึกษาที่หนึ่งปีการศึกษามี

In [55]:
source = texts[10].metadata.get("source")
source = re.sub(r'^.*?\\', '', source)
print(source)

Final_หลักสูตรวท.บ. (วิทยาศาสตร์และนวัตกรรมข้อมูล) (พหุ) (ปป. 66) (2).pdf


In [62]:
content = texts[0].metadata.get("total_pages")
print(content)

140
